# Training Process

In this section, I will tell you how to build and train a neural network which recognizes handwritten digits. We use the MNIST dataset which is a collection of a training set of 60,000 examples, and a test set of 10,000 examples. It is a good database for those who like to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. We will be solving this classification problem with less than 120 lines of Python / TensorFlow / TF-Slim codes.

Our neural network takes in handwritten digits and classifies them, i.e. states if it recognizes them as a 0, a 1, a 2 and so on up to a 9. Each image in the MNIST dataset is a 28x28 pixel greyscale image.

Below is our main code for training the data. I have tried to make it as short and clear as possible. In this workshop, we will be using LeNet as the network. 

So, here we go.

First of all, let's import our required modules. We need to import Tensorflow as the main platform. We also import `MNIST` which is the module for reading data in our desired fashion as well as `load_batch` that loads the data in batches of favorable size. The latter helps to not to go beyond our memory and computational facilities.  


In [ ]:
import tensorflow as tf

from datasets import mnist

from model import lenet, load_batch

I have tried other libraries before like Tensorflow and Keras. Both have their pros and cons, but we will be moving forward with another flexible library called slim. Slim has a good support and a lot of pretrained models like ResNet, VGG, Inception, and others. Actually, slim is a very clean and lightweight wrapper around Tensorflow which will keep our script much neater than using Tensorflow itself which is a very low-level library.

Slim comes with Tensorflow so no need to install it separately. A big advantage, yeah?!

In [ ]:
slim = tf.contrib.slim

Let's facilitate the script by allocating some flags for the data directory, data loading batch size, etc. 

In [ ]:
flags = tf.app.flags
flags.DEFINE_string('data_dir', './data/',
                    'Directory with the mnist data.')
flags.DEFINE_integer('batch_size', 5, 'Batch size.')
flags.DEFINE_integer('num_batches', 200,
                     'Num of batches to train (epochs).')
flags.DEFINE_string('log_dir', './log/train',
                    'Directory with the log data.')
FLAGS = flags.FLAGS

We load the training dataset using `mnist.get_split`. This function has systematic instructions for reading data from `TFRecords`.

In [ ]:
dataset = mnist.get_split('...', FLAGS.data_dir)

Load batches of the dataset. The important arguments for this function are the dataset from the step above and the batch size value which we have already provided through the flags. For this specific workshop, we choose the batch size to be 100. Feel free to change the batch size but be aware of your memory capability! Don't forget that 100 here means 100 images + 100 labels.

In [ ]:
images, labels = load_batch(
    dataset,
    FLAGS.batch_size,
    is_training=True)

Here you see the training digits being fed into the deep neural network, 100 at a time. Then run the batches of images through the neural network model.

In [ ]:
predictions = ...

Then we encode the labels by "One-hot" encoding. This method means that we represent the label "6", for instance, by using a vector of 10 values, all zeros but the 6th value which is 1 (see the image below). 
It is handy here because the format is very similar to how our neural network outputs predictions, also as a vector of 10 values.

In [ ]:
one_hot_labels = slim.one_hot_encoding(
    labels,
    dataset.num_classes)

To drive the training, we will define a loss function, i.e. a value representing how badly the system recognizes the digits and try to minimise it. The statistical practice requires choosing a suitable loss function in the context of a particular applied problem. With `tf.summary` we write the loss values as summaries to be plotted later on Tensorboard.

In [ ]:
tf.losses.softmax_cross_entropy(
    one_hot_labels,
    predictions)

total_loss = tf.losses.get_total_loss()
tf.summary.scalar('loss', total_loss)

This is where the TensorFlow magic happens. You select an optimiser (there are many available such as `GradientDescentOptimizer`, `MomentumOptimizer`, `AdamOptimizer`, etc.) and ask it to minimise the cross-entropy loss. In this step, TensorFlow computes the partial derivatives of the loss function relatively to all the weights and all the biases (the gradient). We need to specify the optimizer. Here, we use `RMSProp` as the optimizer that utilizes the magnitude of recent gradients to normalize the gradients.

In [ ]:
optimizer = tf.train. ...(0.001, 0.9)

Before starting train loop, I should note that our train loop needs a train operation that we call it `train_op`.
This is a crucial `Operation` that:

(a) computes the loss,

(b) applies the gradients to update the weights and

(c) returns the value of the loss. 

`slim.learning.create_train_op` creates such an `Operation`.

In [ ]:
train_op = slim.learning.create_train_op(
    total_loss,
    optimizer,
    summarize_gradients=True)

Finally, it is time to run the training loop. All the TensorFlow instructions up to this point have been preparing a computation graph in memory but nothing has been computed yet. Now in this stage it's better to run evaluation part first. 

"Training" the neural network actually means using training images and labels to adjust weights and biases so as to minimise the cross-entropy loss function. Now, we are all set to start training loop. Use `slim.learning.train` for that. As you see this function uses the operation `train_op` which we have already made in the previous step.

In [ ]:
...(
    train_op,
    FLAGS.log_dir,
    save_summaries_secs=20)